In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import *
import datetime
spark = SparkSession.builder.getOrCreate()

22/03/23 21:42:28 WARN Utils: Your hostname, LAPTOP-V8EALT7T resolves to a loopback address: 127.0.1.1; using 172.20.10.2 instead (on interface wifi0)
22/03/23 21:42:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/23 21:42:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark \
.read \
.option('header',True)\
.option('inferSchema', True)\
.csv(f'/home/dmitry/Business Sales Transaction.csv') \
.createOrReplaceTempView('business_sales')

In [29]:
# Tasks:
# 1. How much money each customer spent per month? 
# 2. How many different items of goods he bought?
# 3. What goods he bought most often?

In [4]:
# create a df
df = spark.table('business_sales')
df.limit(10).toPandas()

,TransactionNo,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country
0,536365,2018-12-01,85123A,Cream Hanging Heart T-Light Holder,1.88,6,17850,United Kingdom
1,536365,2018-12-01,71053,White Moroccan Metal Lantern,2.01,6,17850,United Kingdom
2,536365,2018-12-01,84406B,Cream Cupid Hearts Coat Hanger,1.91,8,17850,United Kingdom
3,536365,2018-12-01,84029G,Knitted Union Flag Hot Water Bottle,2.01,6,17850,United Kingdom
4,536365,2018-12-01,84029E,Red Woolly Hottie White Heart,2.01,6,17850,United Kingdom
5,536365,2018-12-01,22752,Set 7 Babushka Nesting Boxes,2.65,2,17850,United Kingdom
6,536365,2018-12-01,21730,Glass Star Frosted T-Light Holder,2.14,6,17850,United Kingdom
7,536366,2018-12-01,22633,Hand Warmer Union Jack,1.78,6,17850,United Kingdom
8,536366,2018-12-01,22632,Hand Warmer Red Retrospot,1.78,6,17850,United Kingdom
9,536367,2018-12-01,84879,Assorted Colour Bird Ornament,1.75,32,13047,United Kingdom


In [5]:
# rename columns
df = df\
.withColumnRenamed('TransactionNo', 'transaction_id')\
.withColumnRenamed('Date', 'date')\
.withColumnRenamed('ProductNo', 'product_number')\
.withColumnRenamed('ProductName', 'product_name')\
.withColumnRenamed('Price', 'product_price')\
.withColumnRenamed('Quantity', 'quantity')\
.withColumnRenamed('CustomerNo', 'customer_id')\
.withColumnRenamed('Country', 'country')

In [6]:
# drop null rows
df.dropna()

DataFrame[transaction_id: string, date: string, product_number: string, product_name: string, product_price: double, quantity: int, customer_id: string, country: string]

In [7]:
# total price and unique products count for each month and customer_id
df_month_cusid = df\
.withColumn('order_month',F.month(F.col('date')))\
.where(F.col('quantity') > 0)\
.groupBy(F.col('order_month'),F.col('customer_id'))\
.agg(
    F.sum(F.col('product_price') * F.col('quantity')).alias('total_price'),
    F.countDistinct(F.col('product_name')).alias('products_cnt_unique'))\
.withColumn('product_price', F.col('total_price').cast('float'))

#select values
df_month_cusid = df_month_cusid\
.select(F.col('order_month'), F.col('customer_id'), F.col('product_price'), F.col('products_cnt_unique'))\
.orderBy('customer_id','order_month')

df_month_cusid.limit(10).toPandas()

,order_month,customer_id,product_price,products_cnt_unique
0,4,12004,227.139999,56
1,5,12006,3.620000,1
2,3,12008,838.539978,203
3,12,12013,10.230000,1
4,6,12024,21.840000,5
5,2,12025,176.259995,46
6,1,12026,3032.830078,406
7,9,12031,36.200001,7
8,3,12042,929.330017,223
9,7,12043,207.289993,36


In [8]:
# cnt of products by month and customer_id
find_product = df\
.where(F.col('quantity') > 0)\
.withColumn('order_month',F.month(F.col('date')))\
.groupBy(F.col('order_month'),F.col('customer_id'), F.col('product_name'))\
.agg(
    F.sum(F.col('quantity')).alias('product_cnt'))

# show df
find_product\
.orderBy('customer_id','order_month').show(70)

+-----------+-----------+--------------------+-----------+
|order_month|customer_id|        product_name|product_cnt|
+-----------+-----------+--------------------+-----------+
|          4|      12004|Jumbo Bag Scandin...|          1|
|          4|      12004|Childrens Garden ...|          1|
|          4|      12004|Set 3 Retrospot T...|          1|
|          4|      12004|"Charlie+Lola""Ex...|          1|
|          4|      12004|Citronella Candle...|          2|
|          4|      12004|Set/3 Red Gingham...|          2|
|          4|      12004| Photo Frame Cornice|          1|
|          4|      12004|      Popcorn Holder|          2|
|          4|      12004|Strawberry Cerami...|          4|
|          4|      12004|  Red Retrospot Bowl|          1|
|          4|      12004|French Blue Metal...|          1|
|          4|      12004|Gingerbread Man C...|          1|
|          4|      12004|Open Closed Metal...|          1|
|          4|      12004|French Blue Metal...|          

In [10]:
# the most popular product by month and customer_id, over clause
most_popular_product = find_product.orderBy(F.col('product_cnt'), ascending = False)\
.groupBy(F.col('order_month'), F.col('customer_id'))\
.agg(F.first(F.col('product_name')).alias('popular_product_name'))

# popular_product_name by each month and customer_id
most_popular_product\
.orderBy('customer_id','order_month').limit(10).toPandas()

,order_month,customer_id,popular_product_name
0,4,12004,Red Enchanted Forest Placemat
1,5,12006,Jumbo Storage Bag Skulls
2,3,12008,Fluted Antique Candle Holder
3,12,12013,Cabin Bag Vintage Paisley
4,6,12024,Vintage Caravan Gift Wrap
5,2,12025,Jumbo Bag Red Retrospot
6,1,12026,Metal Sign Cupcake Single Hook
7,9,12031,Jumbo Bag Owls
8,3,12042,Black Mini Tape Measure
9,7,12043,Jumbo Shopper Vintage Red Paisley


In [11]:
# join 2 df and create a view of data: 'updated_df'
updated_df = df_month_cusid\
.join(most_popular_product, ['customer_id', 'order_month'], 'outer')\
.select(df_month_cusid.customer_id,
        df_month_cusid.order_month, 
        df_month_cusid.product_price,
        df_month_cusid.products_cnt_unique, 
        most_popular_product.popular_product_name)

updated_df.orderBy('customer_id', 'order_month').limit(20).toPandas()

,customer_id,order_month,product_price,products_cnt_unique,popular_product_name
0,12004,4,227.139999,56,Red Enchanted Forest Placemat
1,12006,5,3.620000,1,Jumbo Storage Bag Skulls
2,12008,3,838.539978,203,Fluted Antique Candle Holder
3,12013,12,10.230000,1,Cabin Bag Vintage Paisley
4,12024,6,21.840000,5,Vintage Caravan Gift Wrap
5,12025,2,176.259995,46,Jumbo Bag Red Retrospot
6,12026,1,3032.830078,406,Metal Sign Cupcake Single Hook
7,12031,9,36.200001,7,Jumbo Bag Owls
8,12042,3,929.330017,223,Black Mini Tape Measure
9,12043,7,207.289993,36,Jumbo Shopper Vintage Red Paisley
